# Diverging network


In [1]:
# fileName

fileName = 'eg_diverging-network' 

In [2]:
# libs

import pyNN.spiNNaker as sim
from pyNN import space 
import numpy as np
import matplotlib.pyplot as plt


Detected PyNN version 0.9.4 and Neo version 0.6.1


In [3]:
# simulation settings

sim.setup(
        timestep=1, # [ms]
        min_delay=1, # [ms]
        max_delay=100) # [ms]

simtime = 250 #ms

2022-01-02 18:10:24 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/spinnaker/.spynnaker.cfg
2022-01-02 18:10:24 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2022-01-02 18:10:24 WARNING: /home/spinnaker/CNT-2021/SpiNNaker/reports has 10 old reports that have not been closed


['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/spinnaker/.spynnaker.cfg']


2022-01-02 18:10:24 WARNING: /home/spinnaker/CNT-2021/SpiNNaker/application_generated_data_files has 11 old reports that have not been closed
2022-01-02 18:10:24 INFO: Setting time scale factor to 1.
2022-01-02 18:10:24 INFO: Setting machine time step to 1000 micro-seconds.


In [4]:
# defining the network

pops = {}
n_cells = {}
neuron_Model = {}
neuron_ModelPars = {}
initial_Values = {}

for popName in ['Exc']:
    n_cells[popName] = 100
    neuron_Model[popName] = sim.IF_cond_exp
    neuron_ModelPars[popName] = sim.IF_cond_exp.default_parameters # or standard_pars or testing_pars
    initial_Values[popName] = sim.IF_cond_exp.default_initial_values
    pops[popName] = sim.Population(n_cells[popName], 
                                   neuron_Model[popName],
                                   neuron_ModelPars[popName],
                                   structure = space.Line(dx=1.0, x0=0.0, y=0.0, z=0.0),
                                   label = popName)
                                   # add spatial constraint

    pops[popName].record(['spikes','v','gsyn_exc','gsyn_inh'])   
 
pops

{'Exc': <spynnaker8.models.populations.population.Population at 0x7fb405106a08>}

In [5]:
# defining the list of connections

def compute_manual_list(n_cells_i, n_cells_j, weights, d_thresh, p_thresh, width): 
        v_c = 0.35 #m/s
        d0 = 1 #ms
        scale = 1

        connections = {}
        probabilities = {}
        distances = {}

        connections = []
        probabilities = []
        distances = []
        for pre in range(n_cells_i):
            for post in range(n_cells_j):
                d_ij = np.sqrt((pre - post)**2)
                delay = d0 + d_ij / v_c
                if d_ij > d_thresh: 
                    distances.append(d_ij)
                    p_ij = scale*np.exp(-0.5 * (d_ij**2/width**2))
                    probabilities.append(p_ij)

                    if p_ij > p_thresh:
                        connections.append([pre, post, weights, delay])#, [d_ij, p_ij]])

        return connections, distances, probabilities


In [6]:
# compute manual list of connections 

weight = {('Exc', 'Exc') : 0.08}
          
d_thresh = {('Exc', 'Exc'): 0}

p_thresh = {('Exc', 'Exc'): 0.2}

width = {('Exc', 'Exc'): 1}



connections = {}
distances = {}
probabilities = {}

connections['Exc', 'Exc'], distances['Exc', 'Exc'], probabilities['Exc', 'Exc'] = compute_manual_list(n_cells['Exc'], 
                                                                                                      n_cells['Exc'], 
                                                                                                      weight['Exc', 'Exc'], 
                                                                                                      d_thresh['Exc', 'Exc'], 
                                                                                                      p_thresh['Exc', 'Exc'], 
                                                                                                      width['Exc', 'Exc'])


In [7]:
# visual check of the connections

fig, ax = plt.subplots(1,2, figsize=(9,5))
fig.tight_layout(pad=3)
axes_list = fig.axes

axes_list[0].plot(np.asarray(connections['Exc', 'Exc']).T[0], np.asarray(connections['Exc', 'Exc']).T[1],'go', label='Exc (i) - Exc (j)')
axes_list[0].grid()
axes_list[0].legend()
axes_list[0].set_title('scatter plot of connections')
axes_list[0].set_xlabel('i cells')
axes_list[0].set_ylabel('j cells')
axes_list[0].set_xlim(20,40)
axes_list[0].set_ylim(20,40)

axes_list[1].plot(distances['Exc', 'Exc'],probabilities['Exc', 'Exc'],'g+')
axes_list[1].plot(np.arange(0, 50), 1*np.exp(-0.5 * (np.arange(0, 50)**2/width['Exc', 'Exc']**2)), 'g:')
axes_list[1].grid()
axes_list[1].set_xlim(0,15)
axes_list[1].set_ylim(0,1)
axes_list[1].axhline(p_thresh['Exc', 'Exc'], color='k', label='p_threshold', )
axes_list[1].set_title('connections as function of prob and dist')
axes_list[1].set_xlabel('distance')
axes_list[1].set_ylabel('probability')
axes_list[1].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# make the projections

proj = {}



proj['Exc', 'Exc'] = sim.Projection(pops['Exc'], 
                                    pops['Exc'],
                                    connector = sim.FromListConnector(connections['Exc', 'Exc']),
                                    receptor_type = 'excitatory',
                                    space = space.Space(axes = 'x'),
                                    label = 'exc-exc connections'
                                    )


In [9]:
# make the thalamic input sources

popName = 'Thalamus'  

n_cells[popName] = n_cells['Exc']
spike_times = [[]]*n_cells[popName] #list of spike lists, where one spike list is related to one spike source
#random_sources_idx = [np.random.randint(n_cells[popName]*0.40, n_cells[popName]*0.60) for i in range(n_cells[popName])]
random_sources_idx = np.arange(int(n_cells[popName]*0.40), int(n_cells[popName]*0.60))

for idx, sources in enumerate(random_sources_idx):
    spike_times[sources] = np.sort([abs(np.random.normal(loc=100, scale=25)) for n in range(5)])


neuron_Model[popName] = sim.SpikeSourceArray(spike_times)


pops[popName] = sim.Population(n_cells[popName], 
                           neuron_Model[popName],
                           label = popName)

proj['Thalamus', 'Exc'] = sim.Projection(pops['Thalamus'], 
                                         pops['Exc'],
                                         connector = sim.OneToOneConnector(),
                                         synapse_type = sim.StaticSynapse(weight=0.08, delay=1),
                                         receptor_type = 'excitatory',
                                         label = 'thalamus-exc connections')

fig, axes = plt.subplots(1,1)
axes_list = fig.axes
axes_list[0].eventplot(spike_times)
axes_list[0].set_xlabel('[ms]')
axes_list[0].set_ylabel('thalamic spike sources')
axes_list[0].set_title('rasterplot of spike sources')
axes_list[0].set_xlim(0, simtime)

random_sources_idx

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
       57, 58, 59])

In [10]:
# run simulation

sim.run(simtime) 

2022-01-02 18:10:25 INFO: Simulating for 250 1.0ms timesteps using a hardware timestep of 1000us
2022-01-02 18:10:25 INFO: Starting execution process
2022-01-02 18:10:28 INFO: Time 0:00:03.112874 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2022-01-02 18:10:36 INFO: Time 0:00:08.500329 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2022-01-02 18:10:41 INFO: Time 0:00:04.817832 taken by PartitionAndPlacePartitioner
Created spalloc job 6226199
2022-01-02 18:10:41 INFO: Created spalloc job 6226199
Waiting for board power commands to complete.
2022-01-02 18:10:41 INFO: Waiting for board power commands to complete.
2022-01-02 18:10:46 INFO: Time 0:00:05.054774 taken by SpallocAllocator
2022-01-02 1

250.0

In [11]:
# save the results
outputs = {}

for layer in ['Exc']:
    outputs[layer] = pops[layer].get_data()
    for recording in ['v', 'gsyn_inh', 'gsyn_exc', 'spikes']:
        pops[layer].write_data(fileName + '_' + str(layer) + '_' + str(recording) + '.pkl')

Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%                         100%|
Getting spikes for Exc
|0%                          50%                         100%|
Getting v for Exc
|0%                          50%                         100%|
Getting gsyn_exc for Exc
|0%                          50%                         100%|
Getting gsyn_inh for Exc
|0%                          50%  

In [12]:
# check results

def recover_results(outputs):
    results = {}
    for key in outputs.keys(): # to extract the name of the layer, e.g., Exc, Inh, Thalamus, etc  
        
        # to get voltage and conductances
        for analogsignal in outputs[key].segments[0].analogsignals:
            print(analogsignal.name)
            results[key, analogsignal.name] = analogsignal

        # to get spikes
        results[key, 'spikes'] = outputs[key].segments[0].spiketrains
    return results

results = recover_results(outputs)
results.keys()

v
gsyn_exc
gsyn_inh


dict_keys([('Exc', 'v'), ('Exc', 'gsyn_exc'), ('Exc', 'gsyn_inh'), ('Exc', 'spikes')])

In [13]:
# check the spikes

fig, axes = plt.subplots(1, 1)
fig.tight_layout(pad=5)
axes_list = fig.axes

for idx, value in enumerate(['Exc']):
    axes_list[idx].eventplot(results[value, 'spikes'])
    axes_list[idx].eventplot(spike_times, color='r')
    axes_list[idx].set_title('rasterplot of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('[ms]')
    axes_list[idx].set_ylabel('cells')
    axes_list[idx].set_xlim(0, simtime)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# check the voltage

fig, axes = plt.subplots(1, 1)#, figsize=(9,5))
fig.tight_layout(pad=4)

axes_list = fig.axes

for idx, value in enumerate(['Exc']):
    im = axes_list[idx].imshow(results[value, 'v'].T)
    axes_list[idx].set_title('voltage of ' + str(value) + ' layer')
    axes_list[idx].set_xlabel('time [ms]')
    axes_list[idx].set_ylabel('cells')
    fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[mV]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# check the conductances

# check the voltage

fig, axes = plt.subplots(2, 1)#, figsize=(9,5))
fig.tight_layout(pad=4)

axes_list = fig.axes

for idx, value in enumerate(['Exc']):
    for idx, gsyn in enumerate(['gsyn_exc', 'gsyn_inh']):
        im = axes_list[idx].imshow(results['Exc', gsyn].T)
        axes_list[idx].set_title(str(gsyn))
        axes_list[idx].set_xlabel('time [ms]')
        axes_list[idx].set_ylabel('cells')
        fig.colorbar(im, ax=axes_list[idx], fraction=0.010, label='[uS]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# end simulation

sim.end()

# task 1: on the stimulus

- go to the notebook cell '# make the thalamic input sources' and design and test a new type of thalamic input (e.g., more/sparser input cells, etc)
